In [4]:
import os
print(os.getcwd())

/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity


In [8]:
import pandas as pd
import numpy as np
import itertools
import os

 
diretorio = "/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity"


file_list = [os.path.join(diretorio, f) for f in os.listdir(diretorio) if f.endswith(".xlsx")]

def calcula_mse(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    df=pd.read_excel(f"1000_model_18_4_14.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    mse = np.mean((Z - Z_media) ** 2 )

    #print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    df=pd.read_excel(f"1000_model_18_4_14.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

def mse_var_bias_covar(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    df=pd.read_excel(f"1000_model_18_4_14.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    var1 = np.mean([(z_pred - Z_media) ** 2 for z_pred in z_preds])
    var = [np.mean((z_pred - Z_media) ** 2) for z_pred in z_preds]
    var = np.mean(np.array(var))

    print(f"var: {var}")
    print(f"var1: {var1}")
    print(var.shape)

    bias=np.mean((Z_media- Z) ** 2 )
    print(f"bias: {bias}")
    
    M = 10  #quantidade de redes
    cov_sum = 0
    for i in range(M):
        for j in range(M):
            if i != j:
                f_i = z_preds[i]
                f_j = z_preds[j]
                f_i_mean = np.mean(f_i)
                f_j_mean = np.mean(f_j)
                cov_ij = np.mean((f_i - f_i_mean) * (f_j - f_j_mean))
                cov_sum += cov_ij

    covar = cov_sum / (M * (M - 1))
    print(f"covar: {covar}")

    return ( var, bias, covar)


def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max,mse_max):
    resultados = []
    
    for count, combinacao in enumerate(itertools.combinations(file_list, tamanho_conjunto), start=1):
        #print(f'Testando combinação {count}: {combinacao}')
        
        mse = calcula_mse(combinacao)
        erro = diversity(combinacao)
        var = mse_var_bias_covar(combinacao)
       

        if  (mse < mse_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro})
            print (f"mse: {mse}")
            salvar_resultados(resultados, pasta_destino1)
        
        if  (erro < erro_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro})
            print (f"ERRO: {erro}")
            salvar_resultados(resultados, pasta_destino)

    return resultados

def salvar_resultados(resultados, pasta_destino):
    
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    for count, resultado in enumerate(resultados, start=1):
        
        nome_arquivo = f"resultado_{count}.xlsx"
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
        
        
        df = pd.DataFrame([resultado])
        
        
        df.to_excel(caminho_arquivo, index=False)
        print(f"Resultado salvo em: {caminho_arquivo}")


# Definição dos parâmetros
tamanho_conjunto = 10
erro_max = 20000  # Limite do erro de diversidade
mse_max = 2000 # Limite do mse_sup
pasta_destino = "resultados_filtrados_erro"
pasta_destino1 = "resultados_filtrados_mse"

# Execução do filtro
resultados_filtrados = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max,mse_max)

var: 216.73685911740466
var1: 216.73685911740466
()
bias: 2176.5981888269644
covar: 384690.3050362692
var: 246.03050328405985
var1: 246.03050328405985
()
bias: 2197.3040672265533
covar: 384296.1589041086


KeyboardInterrupt: 

In [10]:
import pandas as pd
import numpy as np
import itertools
import os

# Diretório onde os arquivos estão armazenados
diretorio = "/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity"

# Lista de arquivos disponíveis no diretório
file_list = [os.path.join(diretorio, f) for f in os.listdir(diretorio) if f.endswith(".xlsx")]


def calcula_mse(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    mse = np.mean([(Z - z_pred) ** 2 for z_pred in z_preds])

    #print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max):
    resultados = []
    
    for count, combinacao in enumerate(itertools.combinations(file_list, tamanho_conjunto), start=1):
        #print(f'Testando combinação {count}: {combinacao}')
        
        mse = calcula_mse(combinacao)
        erro = diversity(combinacao)
        
        if mse is not None and (erro < erro_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro})
            salvar_resultados(resultados, pasta_destino )
        # Limite para evitar consumo excessivo de memória
        if len(resultados) >= 1000:
            break

    return resultados

def salvar_resultados(resultados, pasta_destino):
    # Cria a pasta caso ela não exista
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    for count, resultado in enumerate(resultados, start=1):
        # Nome do arquivo Excel baseado no índice da combinação
        nome_arquivo = f"resultado_{count}.xlsx"
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
        
        # Convertendo o dicionário de resultados para um DataFrame
        df = pd.DataFrame([resultado])
        
        # Salvando o DataFrame no arquivo Excel
        df.to_excel(caminho_arquivo, index=False)
        print(f"Resultado salvo em: {caminho_arquivo}")


# Definição dos parâmetros
tamanho_conjunto = 10
erro_max = 500000  # Limite do erro de diversidade
pasta_destino = "resultados_filtrados"

# Execução do filtro
resultados_filtrados = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max)
#print(resultados_filtrados)

Resultado salvo em: resultados_filtrados/resultado_1.xlsx
Resultado salvo em: resultados_filtrados/resultado_1.xlsx
Resultado salvo em: resultados_filtrados/resultado_2.xlsx
Resultado salvo em: resultados_filtrados/resultado_1.xlsx
Resultado salvo em: resultados_filtrados/resultado_2.xlsx
Resultado salvo em: resultados_filtrados/resultado_3.xlsx


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import itertools
import os
import shutil  

def limpar_pasta(pasta_destino):
    """Remove todos os arquivos dentro da pasta de resultados antes de rodar novamente."""
    if os.path.exists(pasta_destino):
        shutil.rmtree(pasta_destino)  # Remove a pasta inteira
    os.makedirs(pasta_destino)  # Cria uma pasta vazia novamente


limpar_pasta(pasta_destino)

# Diretório onde os arquivos estão armazenados
diretorio = "/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity"

# Lista de arquivos disponíveis no diretório
file_list = [os.path.join(diretorio, f) for f in os.listdir(diretorio) if f.endswith(".xlsx")]


def calcula_mse(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    mse = np.mean([(Z - z_pred) ** 2 for z_pred in z_preds])

    #print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max):
    resultados = []
    
    for count, combinacao in enumerate(itertools.combinations(file_list, tamanho_conjunto), start=1):
        #print(f'Testando combinação {count}: {combinacao}')
        
        mse = calcula_mse(combinacao)
        erro = diversity(combinacao)
        
        if mse is not None and (erro < erro_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro})
            salvar_resultados(resultados, pasta_destino )
        # Limite para evitar consumo excessivo de memória
        if len(resultados) >= 1000:
            break

    return resultados

def salvar_resultados(resultados, pasta_destino):
    # Cria a pasta caso ela não exista
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    for count, resultado in enumerate(resultados, start=1):
        # Nome do arquivo Excel baseado no índice da combinação
        nome_arquivo = f"resultado_{count}.xlsx"
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
        
        # Convertendo o dicionário de resultados para um DataFrame
        df = pd.DataFrame([resultado])
        
        # Salvando o DataFrame no arquivo Excel
        df.to_excel(caminho_arquivo, index=False)
        print(f"Resultado salvo em: {caminho_arquivo}")


# Definição dos parâmetros
tamanho_conjunto = 10
erro_max = 500000  # Limite do erro de diversidade
pasta_destino = "resultados_filtrados"

# Execução do filtro
resultados_filtrados = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max)
#print(resultados_filtrados)



/tmp/ipykernel_17467/2344488770.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


NameError: name 'pasta_destino' is not defined

In [8]:
import pandas as pd
import numpy as np
import itertools
import os

# Diretório onde os arquivos estão armazenados
diretorio = "/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity"

# Lista de arquivos disponíveis no diretório
file_list = [os.path.join(diretorio, f) for f in os.listdir(diretorio) if f.endswith(".xlsx")]

def calcula_mse(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    mse = np.mean([(Z - z_pred) ** 2 for z_pred in z_preds])

    #print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max):
    resultados = []
    
    for count, combinacao in enumerate(itertools.combinations(file_list, tamanho_conjunto), start=1):
        #print(f'Testando combinação {count}: {combinacao}')
        
        mse = calcula_mse(combinacao)
        erro = diversity(combinacao)
        
        if mse is not None and (erro < erro_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro})
        print (resultados)

    return resultados

# Definição dos parâmetros
tamanho_conjunto = 10
erro_max = 500000  # Limite do erro de diversidade

# Execução do filtro
resultados_filtrados = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max)
#print (resultados_filtrados)
print(f"Foram geradas {len(resultados_filtrados)} combinações válidas.")


[{'Conjunto': ('/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_18_4_13.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_24_9_15.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_24_9_13.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_18_4_1.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_14_9_9.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_14_9_22.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_18_4_20.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_37_9_20.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_9_9_4.xlsx', '/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity/1000_model_9_9_0.xlsx'), 'MSE': 44082.0782940

KeyboardInterrupt: 

In [ ]:
import os
print(os.getcwd())  # Mostra o diretório atual onde o script está rodando
print(os.listdir("resultados_filtrados"))

/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity


FileNotFoundError: [Errno 2] No such file or directory: 'resultados_filtrados'

In [ ]:
import pandas as pd
import numpy as np
import itertools
import os

def calcula_mse(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    mse = np.mean([(Z - z_pred) ** 2 for z_pred in z_preds])

    print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2
    print(f'error: {error}')
    return error.T

def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max):
    combinacoes = itertools.combinations(file_list, tamanho_conjunto)
    
    resultados = []
    for conjunto in combinacoes:
        print(f'conjunto: {conjunto}')
        mse = calcula_mse(conjunto)
        erro = diversity(conjunto)
        if np.all(erro < erro_max):  
            resultados.append({"Conjunto": conjunto, "MSE": mse, "Erro de Diversidade": erro})
              
    return resultados

def salvar_resultados_individualmente(resultados):
    """Salva cada conjunto filtrado em arquivos separados dentro da pasta 'resultados_filtrados'."""
    pasta_resultados = "resultados_filtrados"

    # Criar a pasta se não existir
    os.makedirs(pasta_resultados, exist_ok=True)

    for i, resultado in enumerate(resultados):
        nome_arquivo = os.path.join(pasta_resultados, f"resultado_{i+1}.xlsx")
        df_resultado = pd.DataFrame([resultado])
        df_resultado.to_excel(nome_arquivo, index=False)
        print(f'Arquivo {nome_arquivo} salvo com sucesso!')

# Lista de arquivos disponíveis no diretório atual
file_list = [f for f in os.listdir() if f.endswith(".xlsx")]

# Parâmetros
tamanho_conjunto = 10
erro_max = 50000  

# Executa o filtro e obtém os conjuntos válidos
resultados_validos = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max)

# Salva cada resultado em arquivos separados dentro da pasta
salvar_resultados_individualmente(resultados_validos)


conjunto: ('1000_model_18_4_13.xlsx', '1000_model_24_9_15.xlsx', '1000_model_24_9_13.xlsx', '1000_model_18_4_1.xlsx', '1000_model_14_9_9.xlsx', '1000_model_14_9_22.xlsx', '1000_model_18_4_20.xlsx', '1000_model_37_9_20.xlsx', '1000_model_9_9_4.xlsx', '1000_model_9_9_0.xlsx')


mse: 44082.07829401206
error: [[5579372.18465362]
 [5397053.2831256 ]
 [5124570.18518097]
 ...
 [1982262.0850626 ]
 [2180590.65914406]
 [2378039.65667632]]
conjunto: ('1000_model_18_4_13.xlsx', '1000_model_24_9_15.xlsx', '1000_model_24_9_13.xlsx', '1000_model_18_4_1.xlsx', '1000_model_14_9_9.xlsx', '1000_model_14_9_22.xlsx', '1000_model_18_4_20.xlsx', '1000_model_37_9_20.xlsx', '1000_model_9_9_4.xlsx', '1000_model_26_4_3.xlsx')
mse: 44088.3709773979
error: [[5258513.13469329]
 [5159854.17469822]
 [4961093.78575265]
 ...
 [1999467.72301283]
 [2170807.03001592]
 [2329274.55513591]]
conjunto: ('1000_model_18_4_13.xlsx', '1000_model_24_9_15.xlsx', '1000_model_24_9_13.xlsx', '1000_model_18_4_1.xlsx', '1000_model_14_9_9.xlsx', '1000_model_14_9_22.xlsx', '1000_model_18_4_20.xlsx', '1000_model_37_9_20.xlsx', '1000_model_9_9_4.xlsx', '1000_model_22_5_0.xlsx')
mse: 44039.8040967073
error: [[5245829.58758842]
 [5137449.11469022]
 [4929544.83489277]
 ...
 [1973331.24471969]
 [2148419.75988238]
 [2

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import itertools
import os

def calcula_mse(conjunto):

    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    # Calcula o MSE corretamente
    mse = np.mean([(Z - z_pred) ** 2 for z_pred in z_preds])

    print(f"mse: {mse}")
    return mse

def diversity(conjunto):

    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  # Lê os arquivos
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    Z_pred_total = np.hstack(z_preds)
   # Z_media = valores.mean()
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    # Cálculo do erro:
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    #print (f'error1: {error1}')
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    #print (f'error2: {error2}')
    error = error1 + error2
    print (f'error: {error}')
    return error.T



def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max):
    """Gera e filtra combinações de arquivos baseado no MSE e erro específico."""
    combinacoes = itertools.combinations(file_list, tamanho_conjunto)
    
    # Lista para armazenar combinações filtradas
    resultados = []

    # Filtra apenas as combinações que passam nos critérios
    for conjunto in combinacoes:
        print (f'conjunto: {conjunto}')
        mse = calcula_mse(conjunto)
        erro = diversity(conjunto)
        if np.all(erro < erro_max):  # Passa no filtro do erro
            resultados.append({"Conjunto": conjunto, "MSE": mse, "Erro de Diversidade": erro})
              
    return resultados  # Retorna os conjuntos válidos

def salvar_resultados(resultados, nome_arquivo):
    """Salva os resultados em uma única planilha."""
    df_resultado = pd.DataFrame(resultados)
    
    # Salvando o DataFrame em um arquivo Excel
    df_resultado.to_excel(nome_arquivo, index=False)
    print(f'Arquivo {nome_arquivo} salvo com sucesso!')

# Lista de arquivos disponíveis no diretório atual
file_list = [f for f in os.listdir() if f.endswith(".xlsx")]

# Parâmetros
tamanho_conjunto = 10
erro_max = 1000  # Defina um valor para o erro aceitável

# Executa o filtro e obtém os conjuntos válidos
resultados_validos = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max)

# Lista de arquivos disponíveis no diretório atual
file_list = [f for f in os.listdir() if f.endswith(".xlsx")]

# Salva os resultados em uma única planilha
salvar_resultados(resultados_validos, "resultados.xlsx")




conjunto: ('1000_model_18_4_13.xlsx', '1000_model_24_9_15.xlsx', '1000_model_24_9_13.xlsx', '1000_model_18_4_1.xlsx', '1000_model_14_9_9.xlsx', '1000_model_14_9_22.xlsx', '1000_model_18_4_20.xlsx', '1000_model_37_9_20.xlsx', '1000_model_9_9_4.xlsx', '1000_model_9_9_0.xlsx')
mse: 44082.07829401206
error: [[5579372.18465362]
 [5397053.2831256 ]
 [5124570.18518097]
 ...
 [1982262.0850626 ]
 [2180590.65914406]
 [2378039.65667632]]
conjunto: ('1000_model_18_4_13.xlsx', '1000_model_24_9_15.xlsx', '1000_model_24_9_13.xlsx', '1000_model_18_4_1.xlsx', '1000_model_14_9_9.xlsx', '1000_model_14_9_22.xlsx', '1000_model_18_4_20.xlsx', '1000_model_37_9_20.xlsx', '1000_model_9_9_4.xlsx', '1000_model_26_4_3.xlsx')
mse: 44088.3709773979
error: [[5258513.13469329]
 [5159854.17469822]
 [4961093.78575265]
 ...
 [1999467.72301283]
 [2170807.03001592]
 [2329274.55513591]]
conjunto: ('1000_model_18_4_13.xlsx', '1000_model_24_9_15.xlsx', '1000_model_24_9_13.xlsx', '1000_model_18_4_1.xlsx', '1000_model_14_9_9.x

KeyboardInterrupt: 

In [ ]:
#eu quero ler todos os minhas plnailhas e depois conseguir tirar a media de todos os valores

import pandas as pd
import numpy as np
import itertools
import os
from tqdm import tqdm  # Para barra de progresso

def diversity( file_paths):
    """
    Calcula o erro para um conjunto de redes fornecido e salva os resultados em um arquivo Excel.
    """
    dataframes = [pd.read_excel(path) for path in file_paths]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in dataframes]
    Z_pred_total = np.hstack(z_preds)
   # Z_media = valores.mean()
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    #print(f'Z_med: {Z_media}')

    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    # Cálculo do erro
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    #print (f'error1: {error1}')
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    #print (f'error2: {error2}')
    error = error1 + error2
    #print (f'error: {error}')
    return error

def mse(file_paths):
    dataframes = [pd.read_excel(path) for path in file_paths]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in dataframes]
    
    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    
    # Calcula o erro quadrático médio para cada Z_pred
    mse_values = [(Z - z_pred) ** 2 for z_pred in z_preds]
    
    # Concatena todas as diferenças ao quadrado e calcula a média
    mse = np.mean(np.hstack(mse_values))
    
    print(f'MSE: {mse}')
    return mse

# Lista dos arquivos das 100 redes (substitua pelos nomes reais dos arquivos)
file_list = ["1000_model_18_4_13.xlsx", "1000_model_24_9_15.xlsx", "1000_model_24_9_13.xlsx",
    "1000_model_18_4_1.xlsx", "1000_model_14_9_9.xlsx", "1000_model_14_9_22.xlsx",
    "1000_model_18_4_20.xlsx", "1000_model_37_9_20.xlsx", "1000_model_9_9_4.xlsx",
    "1000_model_9_9_0.xlsx"]

diversity(file_list)
mse(file_list)


Z_med: [[3334.06376953]
 [3188.70734863]
 [3039.83764648]
 ...
 [1395.08121338]
 [1494.13549805]
 [1592.76551514]]
error1: [[5528158.00719082]
 [5368071.13184757]
 [5108709.7342431 ]
 ...
 [1877940.90522739]
 [2024486.48263623]
 [2151595.71470411]]
error2: [[ 51214.17746279]
 [ 28982.15127802]
 [ 15860.45093787]
 ...
 [104321.1798352 ]
 [156104.17650783]
 [226443.94197221]]
error: [[5579372.18465362]
 [5397053.2831256 ]
 [5124570.18518097]
 ...
 [1982262.0850626 ]
 [2180590.65914406]
 [2378039.65667632]]
MSE: 44082.07829401207


44082.07829401207

In [ ]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from scipy.spatial import Delaunay


def diversity(fileName, file_path_1,file_path_2,file_path_3,file_path_4,file_path_5,file_path_6,file_path_7, file_path_8,file_path_9,file_path_10):

  
    df_1=pd.read_excel(file_path_1)
    df_2=pd.read_excel(file_path_2)
    df_3=pd.read_excel(file_path_3)
    df_4 = pd.read_excel(file_path_4)
    df_5 = pd.read_excel(file_path_5)
    df_6 = pd.read_excel(file_path_6)
    df_7= pd.read_excel(file_path_7)
    df_8 = pd.read_excel(file_path_8)
    df_9 = pd.read_excel(file_path_9)
    df_10= pd.read_excel(file_path_10)
   
  
    z_pred_1 = df_1['Z_pred'].values
    z_pred_2 = df_2['Z_pred'].values
    z_pred_3 = df_3['Z_pred'].values
    z_pred_4 = df_4['Z_pred'].values
    z_pred_5 = df_5['Z_pred'].values
    z_pred_6 = df_6['Z_pred'].values
    z_pred_7 = df_7['Z_pred'].values
    z_pred_8 = df_8['Z_pred'].values
    z_pred_9 = df_9['Z_pred'].values
    z_pred_10 = df_10['Z_pred'].values
    
    
    Z_pred_1 = z_pred_1.reshape(-1, 1)
    Z_pred_2 = z_pred_2.reshape(-1, 1)
    Z_pred_3 = z_pred_3.reshape(-1, 1)
    Z_pred_4 = z_pred_4.reshape(-1, 1)
    Z_pred_5 = z_pred_5.reshape(-1, 1)
    Z_pred_6 = z_pred_6.reshape(-1, 1)
    Z_pred_7 = z_pred_7.reshape(-1, 1)
    Z_pred_8 = z_pred_8.reshape(-1, 1)
    Z_pred_9 = z_pred_9.reshape(-1, 1)
    Z_pred_10 = z_pred_10.reshape(-1, 1)


    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)
    

    Z_pred_total = np.hstack([Z_pred_1, Z_pred_2, Z_pred_3, Z_pred_4, Z_pred_5,
                           Z_pred_6, Z_pred_7, Z_pred_8, Z_pred_9, Z_pred_10])

    Z_pred = np.mean(Z_pred_total, axis=1, keepdims=True)
    print(f'Z_med: {Z_pred}')
    
    error1= (Z-Z_pred_1)**2 +(Z-Z_pred_2)**2+(Z-Z_pred_3)**2+(Z-Z_pred_4)**2+(Z-Z_pred_5)**2+(Z-Z_pred_6)**2+(Z-Z_pred_7)**2+(Z-Z_pred_8)**2+(Z-Z_pred_9)**2+(Z-Z_pred_10)**2
    error2= (Z_pred_1-Z_pred)**2 +(Z_pred_2-Z_pred)**2+(Z_pred_3-Z_pred)**2+(Z_pred_4-Z_pred)**2+(Z_pred_5-Z_pred)**2+(Z_pred_6-Z_pred)**2+(Z_pred_7-Z_pred)**2+(Z_pred_8-Z_pred)**2+(Z_pred_9-Z_pred)**2+(Z_pred_10-Z_pred)**2
    print (error2)
    error = error1+error2
    
    print(f'Error shape: {error.shape}')  # Verificando formato

    # Criando DataFrame para salvar no Excel
    df_error = pd.DataFrame({'Error': error.flatten()})

    # Salvando a planilha
    df_error.to_excel(f"{fileName}.xlsx", index=False)
    print(f'Arquivo {fileName}.xlsx salvo com sucesso!')

    


diversity= diversity("1000_model_error.xlsx","1000_model_18_4_13.xlsx", "1000_model_24_9_15.xlsx", "1000_model_24_9_13.xlsx",
    "1000_model_18_4_1.xlsx", "1000_model_14_9_9.xlsx", "1000_model_14_9_22.xlsx",
    "1000_model_18_4_20.xlsx", "1000_model_37_9_20.xlsx", "1000_model_9_9_4.xlsx",
    "1000_model_9_9_0.xlsx")

#fig = plot_figure("1000_model_error.xlsx","1000_model_med.xlsx")



Z_med: [[3334.06376953]
 [3188.70734863]
 [3039.83764648]
 ...
 [1395.08121338]
 [1494.13549805]
 [1592.76551514]]
[[ 51214.17746279]
 [ 28982.15127802]
 [ 15860.45093787]
 ...
 [104321.1798352 ]
 [156104.17650783]
 [226443.94197221]]
Error shape: (1024, 1)
Arquivo 1000_model_error.xlsx.xlsx salvo com sucesso!
